In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#hardware
device = 'cuda:0'

from circuit_pruner.utils import load_config

config = load_config('./configs/alexnet_sparse_config.py')
model = config.model.to('cpu')


### quick launch

To get a quick sense for the circuit diagramming tools interactivity, run the lines below. then you should be able to access the tool in your browser at 'http://localhost:8050/'


click a vertex or edge to isolate, click anywhere else in the graph to unisolate

In [ ]:
import torch
circuit_dict = torch.load('quick_start/312_circuit_diagram.pt')
circuit = circuit_dict['model']
dfs = [circuit_dict['edge_df'],circuit_dict['node_df']]

In [ ]:
from circuit_pruner.visualizer.circuit_gui import *

viz_folder = './visualization_cache/demo_diagram/'
port=8050
circuit_diagram = launch_circuit_gui(circuit,None,model,None,dfs=dfs,viz_folder=viz_folder,interactive=True,port=port,device=device)

### visualize your own circuit from scratch

In [ ]:
#feature target

layer = 'features_6'
unit = 301

feature_targets = {layer:[unit]}

In [ ]:
#dataloader

from circuit_pruner.data_loading import rank_image_data
from circuit_pruner.data_loading import single_image_data
from torchvision import transforms
from torch.utils import data


#make 'general' filter dataloader
imnet_kwargs = {'num_workers': 4, 'pin_memory': True, 'sampler':None} if 'cuda' in device else {}


#(this data loader is of 2 random images from each imagenet class)
imnet2_dataloader = data.DataLoader(rank_image_data(config.data_path,
                                            config.preprocess,
                                            label_file_path = config.label_file_path,
                                            class_folders=True),
                                            batch_size=5,
                                            shuffle=False,
                                            **imnet_kwargs)


In [ ]:
#setup model

import torch
import os
from copy import deepcopy
from circuit_pruner.force import setup_net_for_circuit_prune, show_model_layer_names
from circuit_pruner.dissected_Conv2d import dissect_model, set_across_model, set_model_target_node


#we need to 'dissect' the model, which allows us to access kernel-wise activation maps
kernel_rankable_model = dissect_model(deepcopy(model), dissect=True,store_ranks = True, device=device)
kernel_rankable_model.to(device)


set_model_target_node(kernel_rankable_model,layer,unit)
set_across_model(kernel_rankable_model,'rank_field','image')
set_across_model(kernel_rankable_model,'absolute_rank',True)

In [ ]:
#actually extract ranks by running images through model

from circuit_pruner.dissected_Conv2d import get_ranklist_from_dissected_Conv2d_modules, clear_ranks_across_model



clear_ranks_across_model(kernel_rankable_model)

iter_dataloader = iter(imnet2_dataloader)
iters = len(iter_dataloader)

for it in range(iters):

    inputs, label = next(iter_dataloader)
    inputs = inputs.to(device)
    

    kernel_rankable_model.zero_grad()

    try:
        outputs = kernel_rankable_model(inputs)
    except:
        pass

ranks = get_ranklist_from_dissected_Conv2d_modules(kernel_rankable_model, structure='kernels')

In [ ]:
#extract model (large circuit)

from circuit_pruner.extraction import model_ranks_2_circuit_model


sparsity = .9

kernel_pruned_circuit,kernel_mask = model_ranks_2_circuit_model(ranks,sparsity,
                                                           model,feature_targets,device,
                                                           structure='kernels',use_effective_mask=True,
                                                           rank_field='image')


In [ ]:
#visualize large circuit (original feature)

from circuit_pruner.visualizer.featureviz  import featviz_in_recep_field

featviz_in_recep_field(kernel_pruned_circuit,layer,0,margin=0)

In [ ]:
#extract model (small circuit)

from circuit_pruner.extraction import model_ranks_2_circuit_model


sparsity = .02

kernel_pruned_circuit,kernel_mask = model_ranks_2_circuit_model(ranks,sparsity,
                                                           model,feature_targets,device,
                                                           structure='kernels',use_effective_mask=True,
                                                           rank_field='image')


In [ ]:
kernel_pruned_circuit

In [ ]:
featviz_in_recep_field(kernel_pruned_circuit,layer,0,margin=0)

In [ ]:
#launch gui
from circuit_pruner.visualizer.circuit_gui import *
port=8050

circuit_diagram = launch_circuit_gui(kernel_pruned_circuit,kernel_mask,
                                     model,ranks,interactive=True,
                                     port=port,device=device)

### testing

In [212]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [213]:
#hardware
device = 'cuda:0'

from circuit_pruner.utils import load_config

config = load_config('./configs/alexnet_sparse_config.py')
model = config.model.to('cpu')


In [ ]:
#feature target

layer = 'features_6'
unit = 360

feature_targets = {layer:[unit]}

In [ ]:
import os
plotpath = 'plots/%s_%s'%(layer,str(unit))
if not os.path.exists(plotpath):
    os.mkdir(plotpath)

In [ ]:
#dataloader

from circuit_pruner.data_loading import rank_image_data
from circuit_pruner.data_loading import single_image_data
from torchvision import transforms
from torch.utils import data


#make 'general' filter dataloader
imnet_kwargs = {'num_workers': 4, 'pin_memory': True, 'sampler':None} if 'cuda' in device else {}


#(this data loader is of 2 random images from each imagenet class)
imnet2_dataloader = data.DataLoader(rank_image_data(config.data_path,
                                            config.preprocess,
                                            label_file_path = config.label_file_path,
                                            class_folders=True),
                                            batch_size=5,
                                            shuffle=False,
                                            **imnet_kwargs)


In [ ]:
#setup model

import torch
import os
from copy import deepcopy
from circuit_pruner.force import setup_net_for_circuit_prune, show_model_layer_names
from circuit_pruner.dissected_Conv2d import dissect_model, set_across_model, set_model_target_node


#we need to 'dissect' the model, which allows us to access kernel-wise activation maps
kernel_rankable_model = dissect_model(deepcopy(model), dissect=True,store_ranks = True, device=device)
kernel_rankable_model.to(device)


set_model_target_node(kernel_rankable_model,layer,unit)
set_across_model(kernel_rankable_model,'rank_field','image')
set_across_model(kernel_rankable_model,'absolute_rank',True)

In [ ]:
#actually extract ranks by running images through model

from circuit_pruner.dissected_Conv2d import get_ranklist_from_dissected_Conv2d_modules, clear_ranks_across_model



clear_ranks_across_model(kernel_rankable_model)

iter_dataloader = iter(imnet2_dataloader)
iters = len(iter_dataloader)

for it in range(iters):

    inputs, label = next(iter_dataloader)
    inputs = inputs.to(device)
    

    kernel_rankable_model.zero_grad()

    try:
        outputs = kernel_rankable_model(inputs)
    except:
        pass

ranks = get_ranklist_from_dissected_Conv2d_modules(kernel_rankable_model, structure='kernels')

In [ ]:
#extract model (large circuit)

from circuit_pruner.extraction import model_ranks_2_circuit_model


sparsity = .9

kernel_pruned_circuit,kernel_mask = model_ranks_2_circuit_model(ranks,sparsity,
                                                           model,feature_targets,device,
                                                           structure='kernels',use_effective_mask=True,
                                                           rank_field='image')


In [ ]:
#visualize large circuit (original feature)

from circuit_pruner.visualizer.featureviz  import featviz_in_recep_field

featviz_in_recep_field(kernel_pruned_circuit,layer,0,margin=0)

In [ ]:
#extract model (small circuit)

from circuit_pruner.extraction import model_ranks_2_circuit_model


sparsity = .015

kernel_pruned_circuit,kernel_mask = model_ranks_2_circuit_model(ranks,sparsity,
                                                           model,feature_targets,device,
                                                           structure='kernels',use_effective_mask=True,
                                                           rank_field='image')


In [ ]:
featviz_in_recep_field(kernel_pruned_circuit,layer,0,margin=0)

In [ ]:
kernel_pruned_circuit

In [ ]:
from torchvision import models
import torch.nn as nn
from circuit_pruner import root_path


model = models.alexnet(pretrained=True)

model_statedict = torch.load(root_path+'/models/alexnet_sparse_statedict.pt',map_location='cpu')
model.load_state_dict(model_statedict)


In [ ]:
from circuit_pruner.visualizer.utils import *

edge_df = gen_circuit_model_mapping_df(model,kernel_mask,ranks,version = 'edges')
node_df = gen_circuit_model_mapping_df(model,kernel_mask,ranks,version = 'nodes')

dfs = [edge_df,node_df]

In [ ]:
#launch gui
from circuit_pruner.visualizer.circuit_gui import *
port=8050


#viz_folder = '/mnt/data/chris/dropbox/Research-Hamblin/Projects/circuit_pruner/visualization_cache/312_small/'

circuit_diagram = launch_circuit_gui(kernel_pruned_circuit,None,model,None,
                                     dfs=dfs,interactive=False,port=port,device=device)


In [ ]:
circuit_diagram[0].show()

In [ ]:
#circuit_diagram[0].show()
circuit_diagram[0].write_image("plots/small_312_circuit_diagram.jpg") 

In [ ]:
kernel_pruned_circuit

In [ ]:
outputs = []

iter_dataloader = iter(imnet2_dataloader)
iters = len(iter_dataloader)

for it in range(iters):

    inputs, label = next(iter_dataloader)
    inputs = inputs.to(device)
    
    kernel_pruned_circuit.zero_grad()
    output = kernel_pruned_circuit(inputs)
    outputs.append(output[:,0])
    
    
outputs = torch.cat(outputs,dim=0)




In [ ]:
outputs = outputs.detach().cpu().flatten().numpy()

In [ ]:
all_orig_acts = torch.load('/mnt/data/chris/dropbox/Research-Hamblin/Projects/circuit_pruner/target_activations/alexnet_sparse/imagenet_2/orig_activations.pt')


In [ ]:
orig_acts = all_orig_acts['activations']["%s:%s"%(layer,unit)].flatten().numpy()

In [ ]:
import numpy as np

corr = round(np.corrcoef(outputs,orig_acts)[0][1],3)

In [ ]:
corr

In [ ]:
# x and y given as DataFrame columns
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()

# Add first scatter trace with medium sized markers
fig.add_trace(
    go.Scatter(
        mode='markers',
        x=orig_acts,
        y=outputs,
        marker=dict(
            opacity=0.5
        )
    )
)

fig.update_layout(plot_bgcolor="rgba(0,0,0,0)",
                  paper_bgcolor='rgba(0,0,0,0)',
                  xaxis_title="Original Activations",
                  yaxis_title="Circuit Activations")

fig.add_annotation(
        x=int(outputs.max()),
        y=int(orig_acts.max()),
        xref="x",
        yref="y",
        text="R^2 = %s"%corr)

fig.show()

In [ ]:
fig.write_image("plots/small_312_circuit_v_orig_scatter.png") 

In [ ]:
from circuit_pruner.visualizer.utils import get_model_conv_weights

In [ ]:
kernels = get_model_conv_weights(kernel_pruned_circuit)

In [ ]:
torch.max(torch.abs(torch.max(kernels[1])),torch.abs(torch.min(kernels[1])))

In [ ]:
model

In [ ]:
from circuit_pruner.visualizer.featureviz  import featviz_in_recep_field
from lucent_video.optvis import render

#config = load_config('./configs/alexnet_sparse_config.py')

#model = config.model.to('cpu')
#model.zero_grad()

#dis_model = dissect_model(model, dissect=True)
#_ = dis_model.to(device).eval()


param_f = lambda: param.image(224)
obj  = objectives.neuron('features_3',176)
_ = render.render_vis(dis_model, obj, param_f,show_inline=True)
#featviz_in_recep_field(kernel_pruned_circuit,layer,0,margin=0)

In [ ]:
output = render.render_vis(dis_model, obj, param_f,show_inline=True)

In [ ]:
#image = output['images']
#params= output['params']

params = params.requires_grad_(False)
#desaturate
params = (params-torch.mean(params))/torch.std(params)*.1
params = params.requires_grad_(True)

param_f = lambda: param.image(224,start_params=params)


obj  = objectives.neuron('features_3',2)

In [ ]:
output = render.render_vis(kernel_pruned_circuit, obj, param_f,show_inline=True,)

In [ ]:
output_file_name = '/mnt/data/chris/dropbox/Research-Hamblin/Projects/circuit_pruner/visualization_cache/circuit_at_1667703040.4020417/featviz/176_features_3:176_1667703090.0236857.jpg'

In [ ]:
from circuit_pruner.receptive_fields import receptive_field, receptive_field_for_unit
from math import floor

margin = 10
all_recep_field_params = receptive_field(kernel_pruned_circuit,(3,224,224))
layer_ind = str(int(layer.replace('features_',''))+1)
map_size = (all_recep_field_params[layer_ind]['output_shape'][2],all_recep_field_params[layer_ind]['output_shape'][3])

print(map_size)
if map_size[0]%2 == 0:
    target_position = (int(map_size[0]/2+1),int(map_size[1]/2+1))
else:
    target_position = (int(floor(map_size[0]/2)),int(floor(map_size[1]/2)))
    
print(target_position)
recep_field = receptive_field_for_unit(all_recep_field_params, layer, target_position)
print(recep_field)


viz_im = output['images'][0][:,int(recep_field[0][0])-margin:int(recep_field[0][1])+margin,int(recep_field[1][0])-margin:int(recep_field[1][1])+margin,:]
#viz_im = viz_im[0][:,:,:,:]
image = (viz_im * 255).astype(np.uint8)
if len(image.shape) == 4:
    image = np.concatenate(image, axis=1)
image = Image.fromarray(image)



image.save(output_file_name)



In [ ]:
layer

In [ ]:
def featviz_in_recep_field(model,feature_name,unit, input_dim = (3,224,224),image_name=None, margin = 10, device = 'cuda:0'):
	from circuit_pruner.receptive_fields import receptive_field, receptive_field_for_unit
	from math import ceil, floor
	from lucent_video.optvis import render, param, transform, objectives
	from copy import deepcopy
	from circuit_pruner.dissected_Conv2d import dissect_model
	from math import floor
	import numpy as np

	dis_model = dissect_model(deepcopy(model), store_ranks = False, device=device)
	dis_model = dis_model.to(device).eval()

	#all_recep_field_params = receptive_field(model.features, input_dim)
	try:
		all_recep_field_params = receptive_field(model.to(device), input_dim)
	except:
		all_recep_field_params = receptive_field(model.features.to(device), input_dim)
	
	param_f = lambda: param.image(input_dim[1])
	obj  = objectives.neuron(feature_name,unit)
	viz_im = render.render_vis(dis_model,  obj, param_f, show_inline=True)
	
	layer_ind = str(int(feature_name.replace('features_',''))+1)
	map_size = (all_recep_field_params[layer_ind]['output_shape'][2],all_recep_field_params[layer_ind]['output_shape'][3])
	print(map_size)
	if map_size[0]%2 == 0:
		target_position = (int(map_size[0]/2+1),int(map_size[1]/2+1))
	else:
		target_position = (int(floor(map_size[0]/2)),int(floor(map_size[1]/2)))
	print(target_position)
	recep_field = receptive_field_for_unit(all_recep_field_params, feature_name, target_position)
	print(recep_field)
	viz_im = viz_im['images'][0][:,int(recep_field[0][0])-margin:int(recep_field[0][1])+margin,int(recep_field[1][0])-margin:int(recep_field[1][1])+margin,:]
	#viz_im = viz_im[0][:,:,:,:]
	image = (viz_im * 255).astype(np.uint8)
	if len(image.shape) == 4:
		image = np.concatenate(image, axis=1)
	image = Image.fromarray(image)

	if image_name is None:
		return image
	else:
		image.save(image_name)
	


In [ ]:
kernel_pruned_circuit